In [5]:
import numpy as np
import tensorflow as tf
from keras.utils.np_utils import to_categorical

mnist = tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
y_train = to_categorical(y_train, 10)
x_train = x_train/255
x_test = x_test/255

print(x_train.shape)
print(y_train.shape)


Using TensorFlow backend.


(60000, 784)
(60000, 10)


In [6]:
class Network:
      
    # A1 = 64*784, b1 = 64, A2 = 10x64, b2 = 10
    def __init__ (self, x_train, y_train, A1=None, b1=None, A2 = None, b2 = None):
        self.A1 = A1
        self.b1 = b1
        self.A2 = A2
        self.b2 = b2
        self.x_train = x_train
        self.y_train = y_train

    def random_weights(self):
        self.A1 = np.random.normal(0, 1, (64, 784))
        self.b1 = np.random.normal(0, 1, 64)
        self.A2 = np.random.normal(0, 1, (10, 64))
        self.b2 = np.random.normal(0, 1, 10)   
    
    def classify(self,x):
        
        x = Network.sigmoidal(self.A1.dot(x)+self.b1)
        y_pred = Network.softmax(self.A2.dot(x)+self.b2)
        
        return np.argmax(y_pred)
    
    def fitness_val(self):
        
        loss = 0
        for i, x in enumerate(x_train):
            x = Network.sigmoidal(self.A1.dot(x)+self.b1)
            y_pred = Network.softmax(self.A2.dot(x)+self.b2)
            
            y_true = y_train[i]
            loss += 1/2*np.sum((y_true-y_pred)**2)
        return 1/(loss/x_train.shape[0])

    def sigmoidal(x):
        return 1/(1+(np.exp(-x)))
    
    def softmax(x):
        return np.exp(x)/np.sum(np.exp(x))
               
               

In [20]:
def produce_child(network1, network2):
    
    A1 = np.zeros((64, 784))
    b1 = np.zeros(64)
    A2 = np.zeros((10,64))
    b2 = np.zeros(10)
            
    for i in range(64):
        b1[i] = np.copy(network1.b1[i]) if np.random.rand(1) < 0.5 else np.copy(network2.b1[i])
        A1[i] = np.copy(network1.A1[i]) if np.random.rand(1) <= 0.5 else np.copy(network2.A1[i])
        if np.random.rand(1) <= 0.5:
            A1[i] += np.random.normal(0, 1.5, 784)
            b1[i] += np.random.normal(0, 1.5, 1)
    
    for i in range(10):
        b2[i] = np.copy(network1.b2[i]) if np.random.rand(1) < 0.5 else np.copy(network2.b2[i])
        A2[i] = np.copy(network1.A2[i]) if np.random.rand(1) <= 0.5 else np.copy(network2.A2[i])
        if np.random.rand(1) <= 0.5:
            A2[i] += np.random.normal(0, 1.5, 64)
            b2[i] += np.random.normal(0, 1.5, 1)
        
    return Network(x_train, y_train, A1, b1, A2, b2)
    
def crossover_and_mutate(network1, network2):
   
    if np.random.rand(1) <= 0.1:
        return (network1, network2)
    
    return (produce_child(network1, network2), produce_child(network1, network2))

In [21]:
##we use population size 14 and 1 elite individuals with 2 copies

gen_old = []
gen_old_fitvals = []

for i in range(14):
    network = Network(x_train, y_train)
    network.random_weights()
    gen_old += [network]
    gen_old_fitvals += [network.fitness_val()]

#start to generate new and better populations
for i in range(75):
    print(i)
    gen_new = []
    gen_new_fitvals = []
    
    most_fit = np.argmax(gen_old_fitvals)
    print(gen_old_fitvals[most_fit])
    gen_new += [gen_old[most_fit], gen_old[most_fit]]
    gen_new_fitvals += [gen_old_fitvals[most_fit], gen_old_fitvals[most_fit]]
    
    for j in range(1, 7):
        print(np.array(gen_old_fitvals)/sum(gen_old_fitvals))
        pair = np.random.choice(14, 2, p = np.array(gen_old_fitvals)/sum(gen_old_fitvals), replace=False)
        print(pair)
        print("***********")
        (network1, network2) = crossover_and_mutate(gen_old[pair[0]], gen_old[pair[1]])
        gen_new += [network1, network2]
        gen_new_fitvals += [network1.fitness_val(), network2.fitness_val()]
        
    gen_old = gen_new
    gen_old_fitvals = gen_new_fitvals
          

KeyboardInterrupt: 

0
1.8299562098229019
[0.08589482 0.08589482 0.06210016 0.0650122  0.06653124 0.0719268
 0.07017416 0.07734647 0.07016613 0.07088438 0.06961718 0.06136984
 0.06402712 0.07905468]
[12  0]
***********
[0.08589482 0.08589482 0.06210016 0.0650122  0.06653124 0.0719268
 0.07017416 0.07734647 0.07016613 0.07088438 0.06961718 0.06136984
 0.06402712 0.07905468]
[ 2 12]
***********
[0.08589482 0.08589482 0.06210016 0.0650122  0.06653124 0.0719268
 0.07017416 0.07734647 0.07016613 0.07088438 0.06961718 0.06136984
 0.06402712 0.07905468]
[12  5]
***********
[0.08589482 0.08589482 0.06210016 0.0650122  0.06653124 0.0719268
 0.07017416 0.07734647 0.07016613 0.07088438 0.06961718 0.06136984
 0.06402712 0.07905468]
[6 1]
***********
[0.08589482 0.08589482 0.06210016 0.0650122  0.06653124 0.0719268
 0.07017416 0.07734647 0.07016613 0.07088438 0.06961718 0.06136984
 0.06402712 0.07905468]
[11 13]
***********
[0.08589482 0.08589482 0.06210016 0.0650122  0.06653124 0.0719268
 0.07017416 0.07734647 0.07016

[0.09270379 0.09270379 0.0653406  0.06381521 0.06238259 0.06829813
 0.09270379 0.06297231 0.07286688 0.06767708 0.06813698 0.06556919
 0.06128735 0.06354231]
[12  4]
***********
[0.09270379 0.09270379 0.0653406  0.06381521 0.06238259 0.06829813
 0.09270379 0.06297231 0.07286688 0.06767708 0.06813698 0.06556919
 0.06128735 0.06354231]
[2 7]
***********
8
1.8299562098229019
[0.09633813 0.09633813 0.07033028 0.09633813 0.06399579 0.07221675
 0.06641749 0.06340446 0.05963425 0.06408429 0.05999664 0.06038815
 0.06495503 0.06556249]
[10  3]
***********
[0.09633813 0.09633813 0.07033028 0.09633813 0.06399579 0.07221675
 0.06641749 0.06340446 0.05963425 0.06408429 0.05999664 0.06038815
 0.06495503 0.06556249]
[ 2 11]
***********
[0.09633813 0.09633813 0.07033028 0.09633813 0.06399579 0.07221675
 0.06641749 0.06340446 0.05963425 0.06408429 0.05999664 0.06038815
 0.06495503 0.06556249]
[0 2]
***********
[0.09633813 0.09633813 0.07033028 0.09633813 0.06399579 0.07221675
 0.06641749 0.06340446 0.0

In [17]:
network = gen_old[np.argmax(gen_old_fitvals)]
acc = 0
for i, x in enumerate(x_test):
    acc += 1 if network.classify(x) == y_test[i] else 0
acc /= x_test.shape[0]


In [18]:
acc

0.3105

[ 1.98303446 -0.84914222  0.06894824 -2.04720467  0.44381904  1.55596649
 -1.51128677  0.73067417  1.32038913  1.53891263  0.0448115   0.59518325
 -1.59871909 -2.29941707  2.35039    -0.35056194 -0.73285228  0.07859675
  0.30362516 -1.2485311  -0.11580739  1.84022662 -0.67331452  1.16207603
  0.87659048  2.10790469 -0.6492751  -0.69744    -0.71408075  0.557835
  0.57436947  1.17940289 -1.24219093 -0.94504688  0.95688369 -2.60630967
  0.73385688 -3.84974976  0.36504759  1.10220931 -1.14981335 -0.32848479
 -1.99827556 -2.02551009  0.12990452 -0.29412493 -2.02231418  1.53192262
 -0.46760259 -0.4064333  -0.00542503  0.42167545  2.80314889  0.6294337
  1.29111135 -0.05257251 -0.18654363 -0.75437328 -0.79203135 -0.10797101
  0.77716823  0.25038143 -1.10700774 -0.09336249]
